<a href="https://colab.research.google.com/github/hyang0129/NGAFIDDATASET/blob/main/NGAFID_DATASET_DASK_EXAMPLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT DEPENDENCIES

In [ ]:
import sys
sys.path.append('/content/NGAFIDDATASET')

In [ ]:
import dask.dataframe as dd
import pandas as pd
flight_data_df = dd.read_parquet('all_flights/one_parq')

flight_header_df = pd.read_csv('all_flights/flight_header.csv', index_col = 'Master Index')


In [ ]:
flight_data_df = flight_data_df.compute()

In [ ]:
flight_data_df.head()

In [ ]:
label_distribution = flight_header_df['label'].value_counts()
label_distribution

In [ ]:
filtered_damage_rows

In [ ]:
damage_rows = flight_header_df[flight_header_df['label'].str.endswith('oil leak/pressure issue ')]
filtered_damage_rows = damage_rows[damage_rows['date_diff'].isin([-2,-1, 1,2])]
a = filtered_damage_rows['date_diff'].value_counts()
a

In [ ]:
master_indices = filtered_damage_rows.index.tolist()
master_indices_list = list(master_indices)
filtered_df = flight_data_df.loc[master_indices_list]
filtered_df.head()

In [ ]:
filtered_df

In [ ]:
target_series = filtered_damage_rows['date_diff'].apply(lambda x: 1 if x in [-3, -2, -1] else 0)
target_series.name = 'target'

# Now join this target column onto sorted_df by their shared index (Master_index)
# Since filtered_damage_rows and sorted_df share these master indices, we can join on index:
sorted_df = filtered_df.join(target_series, how='left')

# If there are any indices in sorted_df not present in filtered_damage_rows (unlikely here),
# they would get NaN. We can fill them with 0 just to be safe:
sorted_df['target'] = sorted_df['target'].fillna(0)

# Now sorted_df has the target column as requested.
print(sorted_df.head())

In [ ]:
# Convert master_indices to a list if not already
master_indices_list = list(master_indices)

# Filter using the Dask DataFrame's query capability
filtered_flight_data = flight_data_df[flight_data_df.index.isin(master_indices_list)]

# Sort by 'timestep' after computing the result
sorted_flight_data = filtered_flight_data.compute().sort_values(by='timestep')

# Display the resulting DataFrame
print(sorted_flight_data)

In [ ]:
target_series = filtered_damage_rows['date_diff'].apply(lambda x: 1 if x in [-3, -2, -1] else 0)
target_series.name = 'target'

# Now join this target column onto sorted_df by their shared index (Master_index)
# Since filtered_damage_rows and sorted_df share these master indices, we can join on index:
sorted_df = sorted_df.join(target_series, how='left')

# If there are any indices in sorted_df not present in filtered_damage_rows (unlikely here),
# they would get NaN. We can fill them with 0 just to be safe:
sorted_df['target'] = sorted_df['target'].fillna(0)

# Now sorted_df has the target column as requested.
print(sorted_df.head())

In [ ]:
# Export to CSV
sorted_df.to_csv('data.csv', index=False)
